# Setup

## Install dependencies

In [1]:
!pip install -qqq langchain-openai langchain-core langchain-community langgraph faiss-cpu

^C
Traceback (most recent call last):
  File "/Users/roopesh/.pyenv/versions/3.11.6/bin/pip", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/Users/roopesh/.pyenv/versions/3.11.6/lib/python3.11/site-packages/pip/_internal/cli/main.py", line 79, in main
    return command.main(cmd_args)
           ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/roopesh/.pyenv/versions/3.11.6/lib/python3.11/site-packages/pip/_internal/cli/base_command.py", line 101, in main
    return self._main(args)
           ^^^^^^^^^^^^^^^^
  File "/Users/roopesh/.pyenv/versions/3.11.6/lib/python3.11/site-packages/pip/_internal/cli/base_command.py", line 236, in _main
    self.handle_pip_version_check(options)
  File "/Users/roopesh/.pyenv/versions/3.11.6/lib/python3.11/site-packages/pip/_internal/cli/req_command.py", line 180, in handle_pip_version_check
    session = self._build_session(
              ^^^^^^^^^^^^^^^^^^^^
  File "/Users/roopesh/.pyenv/versions/3.11.6/lib/python3.11/site-packages/pip/_i

## Create reuqired files

In [9]:
hr_policy_text = """
Leave Policy:
- Annual Leave: 18 days per year. Carry forward up to 6 days.
- Sick Leave: 10 days per year. Medical certificate required beyond 2 days.
- Casual Leave: 6 days per year.

Attendance Policy:
- Working hours: 9:30 AM to 6:30 PM.
- Minimum 8 working hours per day.
- 3 late arrivals per month allowed.

Remote Work Policy:
- Eligible after 6 months tenure.
- Max 2 days per week.
- Not allowed during notice period.

Holiday Policy:
- 10 fixed company holidays per year.

Code of Conduct:
- Professional behavior mandatory.
- No harassment or misuse of company resources.

Exit Policy:
- Notice period: 60 days.
- Buyout subject to HR approval.
"""



In [ ]:
file_path = "/content/hr_policies.txt"

with open(file_path, "w") as f:
    f.write(hr_policy_text)

print("File created at:", file_path)


File created at: hr_policies.txt


# Prepare Data

In [ ]:
# Load the Document
from langchain_community.document_loaders import TextLoader

loader = TextLoader(file_path)
documents = loader.load()

documents[0].page_content[:300]


In [ ]:
# Split Document into Chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=50
)

chunks = text_splitter.split_documents(documents)
len(chunks)


In [ ]:
# Create Embeddings & Vector Store (FAISS)
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(chunks, embeddings)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})



# Tools

In [ ]:
# Define HR Tools
from langchain_core.tools import tool
from datetime import datetime

@tool
def classify_hr_policy(query: str) -> str:
    """Classify HR question into policy category."""
    q = query.lower()
    if "leave" in q:
        return "Leave Policy"
    if "remote" in q or "work from home" in q:
        return "Remote Work Policy"
    if "late" in q or "attendance" in q:
        return "Attendance Policy"
    if "notice" in q or "resign" in q:
        return "Exit Policy"
    return "General HR Policy"


@tool
def calculate_days(start_date: str, end_date: str) -> str:
    """Calculate number of days between two dates (YYYY-MM-DD)."""
    s = datetime.strptime(start_date, "%Y-%m-%d")
    e = datetime.strptime(end_date, "%Y-%m-%d")
    return f"{(e - s).days} days"



In [ ]:
from langchain_core.prompts import ChatPromptTemplate

hr_prompt = ChatPromptTemplate.from_messages([
    ("system",
     """
You are an HR Policy Assistant.

Strict rules:
- Answer ONLY using the provided HR policy context.
- Do NOT guess or invent policies.
- If information is insufficient, ask a clarification question.
- If policy does not define something, say: "Not defined in policy".
- Maintain a professional HR tone.

Always respond using this structure:

Answer:
Source Policy:
Notes:
Confidence Level:
"""),
    ("human", "{question}")
])


# RAG

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

rag_chain = (
    RunnableParallel({
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    })
    | ChatPromptTemplate.from_messages([
        ("system", "Use the following HR policy context:\n\n{context}"),
        ("human", "{question}")
    ])
    | llm
    | StrOutputParser()
)


In [ ]:
# Create agent
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model=llm,
    tools=[classify_hr_policy, calculate_days],
    prompt=hr_prompt
)


In [ ]:
# Ask a Normal HR Question
query = "How many annual leave days do I get?"

result = agent.invoke({
    "messages": [{"role": "user", "content": query}]
})

for msg in result["messages"]:
    if hasattr(msg, "content"):
        print(msg.content)


In [ ]:
# Clarification Example
query = "Can I apply for remote work?"

result = agent.invoke({
    "messages": [{"role": "user", "content": query}]
})

for msg in result["messages"]:
    if hasattr(msg, "content"):
        print(msg.content)


In [ ]:
# Out-of-Scope Question
query = "How is my salary calculated?"

result = agent.invoke({
    "messages": [{"role": "user", "content": query}]
})

for msg in result["messages"]:
    if hasattr(msg, "content"):
        print(msg.content)
